<span style="display:block;text-align:center;margin-right:105px"><img src="../media/logos/hacks-logo.png" width="200"/></span>

## Table of Contents

<ul>
    <li><a href='#Dependencies'>Dependencies</a></li>
    </li>
    <li><a href='#Modelling'>Modelling</a>
        <ol style='margin-top: 0em;'>
            <li><a href='#1.-State-Variables'>State Variables</a></li>
            <li><a href='#2.-System-Parameters'>System Parameters</a></li>
            <li><a href='#3.-Policy-Functions'>Policy Functions</a></li>
            <li><a href='#4.-State-Update-Functions'>State Update Functions</a></li>
            <li><a href='#5.-Partial-State-Update-Blocks'>Partial State Update Blocks</a></li>
        </ol>
    </li>
    <li><a href='#Simulation'>Simulation</a>
        <ol style='margin-top: 0em;' start="6">
            <li><a href='#6.-Configuration'>Configuration</a></li>
            <li><a href='#7.-Execution'>Execution</a></li>
            <li><a href='#8.-Output-Preparation'>Output Preparation</a></li>
            <li><a href='#9.-Analysis'>Analysis</a></li>
        </ol>
    </li>
</ul>

---

# Dependencies

In [1]:
# cadCAD standard dependencies

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

# cadCAD global simulation configuration list
from cadCAD import configs

# Included with cadCAD
import pandas as pd

# Additional dependencies

# For analytics
import numpy as np
rng = np.random.default_rng(42)

# For visualization
import panel as pn
import plotly.express as px
pd.options.plotting.backend = "plotly"

# From our model
from parameterized.proposal_inverter import Wallet
from parameterized.whitelist_mechanism import NoVote

# Modelling

* Have a pool of the agents, like a dictionairy
* Have a pool of proposals, like a dictionairy
* Track the connections between the agents and the PI

The wallets are initialized with random funds.   
Let's start the simulation with 1,000,000 Funds, 100 wallets, 0 owners, 0 proposals, 0 brokers, 0 payers.  
Scenario 1: Largest 20% of wallets become owners.    
Scenario 2: All wallets have an equal chance of becoming owners.

Some wallets will become owners  
1,000,000 Funds, 100 Wallets, 10 Owners, 10 Proposal, 0 Brokers, 0 Payers

Some Brokers Join Proposals and some Payers join proposals  
1,000,000 Funds, 100 Wallets, 10 Owners, 10 Proposal, 7 Brokers, 5 Payers

On every step of the simulation, as an agent (Wallet) I have three considerations to make (3-action space)
1. Become an Owner and start a new Proposal
2. For each proposal, consider if I want to become a payer
3. For each proposal, consider if I want to become a broker
4. If I am a broker, I can make a claim on the proposals that I am a broker on
5. If I am an owner, I can cancel a proposal

Note for implementation. Pools of agents and wallets to be 'global' outside of the simulation state, and referenced as we go. 

## 1. State Variables

The simulation maintains a constant number of funds in the economy and a constant number of wallets that act as agents. Each wallet begins with a random amount of funds that have been distributed from the total.

Over time, the agents will deploy, contribute to, or join proposals, and this will increase the number of owners, payers, and brokers, respectively.

### 1.1 Initial State

In [2]:
initial_state = {
    # Initial conditions of the system
    'total_funds': 500_000,
    "wallet_funds": 500_000,
    "proposal_funds": 0,
    "n_wallets": 50,
    "n_proposals": 0,
    
    # Distribution of agent actions
    'owners': 0,
    'payers': 0,
    'brokers': 0,
    
    # Proposal properties
    "funds_staked": 0,
    "funds_claimed": 0,
    "funds_contributed": 0,
}

### 1.2 Global Variables

In [3]:
# These are the global variables representing the pool of wallets and pool of proposals
WALLETS = dict()
PROPOSALS = dict()
TRANSACTIONS = list() # columns=["timestep", "wallet", "wallet_funds", "action", "proposal", "proposal_funds"]

funds_distribution = rng.random(size=initial_state["n_wallets"])
funds_distribution = funds_distribution / np.sum(funds_distribution) * initial_state["total_funds"]

for funds in funds_distribution:
    wallet = Wallet(funds=funds)
    wallet.feature_vector = rng.uniform(size=wallet.number_of_features)
    WALLETS[wallet.public] = wallet
    
    TRANSACTIONS.append({
        "timestep": 0,
        "wallet": wallet.public,
        "wallet_funds": wallet.funds,
        "action": "initialize",
    })

## 2. System Parameters

Each of the actions and the hyperparameters of each action can be represented as a function that samples from some probability distribution.

- For the actions, the function should return a boolean, indicating whether or not an agent will perform this action during the current epoch.
- For the hyperparameters, the functions should return the value of the hyperparameter.

In [4]:
def feature_noise(n_features=5):
    return rng.normal(loc=0, scale=0.1, size=5)

### 2.1 Action Space Functions

Each action space function represents an agent's decision to perform an action. They should return `True` if the agent decides to perform the action, and `False` otherwise.

These functions are called in the policy functions, but kept separate for easy plug and play for different agent behaviours.

In [5]:
def a_static_0_percent(wallet: Wallet, n_proposals: int) -> list[bool]:
    """Returns `True` with a probability of 0%. In other words, `False` with a probability of 100%."""
    return np.full(shape=n_proposals, fill_value=False)

def a_static_1_percent(wallet: Wallet, n_proposals: int) -> list[bool]:
    """Returns `True` with a probability of 1%."""
    return rng.uniform(size=n_proposals) < 0.01

def a_static_50_percent(wallet: Wallet, n_proposals: int) -> list[bool]:
    """Returns `True` with a probability of 50%."""
    return rng.uniform(size=n_proposals) < 0.5

def a_static_100_percent(wallet: Wallet, n_proposals: int) -> list[bool]:
    """Returns `True` with a probability of 100%."""
    return np.full(shape=n_proposals, fill_value=True)

def a_dynamic_joined(wallet: Wallet, n_proposals: int, pr: float) -> list[bool]:
    """Returns `True` with a specified probability for all proposals this wallet has joined."""
    return np.array([rng.random() < pr if wallet.public in proposal.broker_agreements.keys() else False for proposal in PROPOSALS.values()])

def a_dynamic_funded(wallet: Wallet, n_proposals: int, pr: float) -> list[bool]:
    """Returns `True` with a specified probability for all proposals this wallet has funded."""
    return np.array([rng.random() < pr if wallet.public in proposal.payer_agreements.keys() else False for proposal in PROPOSALS.values()])

def a_decreasing_linear(wallet: Wallet, n_proposals: int, y_scale: float=1) -> list[bool]:
    """Based on a linear distribution where lower funds means a higher probability of joining."""
    max_funds = np.max([wallet.funds for wallet in WALLETS.values()])
    pr = y_scale * (-wallet.funds / max_funds + 1)
    
    return rng.uniform(size=n_proposals) < pr

def a_increasing_linear(wallet: Wallet, n_proposals: int, y_scale: float=1) -> list[bool]:
    """Based on a linear distribution where higher funds means a higher probability of paying."""
    max_funds = np.max([wallet.funds for wallet in WALLETS.values()])
    pr = y_scale * (wallet.funds / max_funds)
    
    return rng.uniform(size=n_proposals) < pr

def a_normal(wallet: Wallet, n_proposals: int, y_scale: float=1) -> list[bool]:
    """Based on a normal Gaussian distribution with the mean funds as the centre."""
    mean_funds = np.mean([wallet.funds for wallet in WALLETS.values()])
    max_funds = np.max([wallet.funds for wallet in WALLETS.values()])
    pr = y_scale * np.exp(-0.5 * np.power((wallet.funds - mean_funds) / (max_funds / 3), 2))
    
    return rng.uniform(size=n_proposals) < pr

def a_probability(wallet: Wallet, n_proposals: int, y_scale: float=1) -> list[bool]:
    """Based on the highest probability from the result of matrix factorization."""
    p_matrix = [proposal.feature_vector for proposal in PROPOSALS.values() if not proposal.cancelled]
    prs = np.matmul(p_matrix, wallet.feature_vector)
    
    return rng.uniform(size=n_proposals) < prs

### 2.2 Hyperparameter Functions

The hyperparameter functions represent the specifics of each action the agent may perform. For example, if the agent decides to join a proposal, then the agent must also decide how much to stake. A hypyerparameter function is then defined to decide how much to stake.

These functions are called in the policy functions, but kept separate for easy plug and play for different agent behaviours.

In [6]:
def h_join_stake(wallet: Wallet) -> float:
    return 10

def h_pay_contribution(wallet: Wallet) -> float:
    return 50

def h_vote_broker(wallet: Wallet) -> int:
    return rng.choice(list(WALLETS.keys()))

def h_vote_result(wallet: Wallet) -> bool:
    return True

def h_deploy_initial_funds(wallet: Wallet) -> float:
    return 100

### 2.3 System Parameters

In [7]:
system_params = {
    "feature_noise": [feature_noise],
    
    # Probabilities defining the action space
    "join": [a_decreasing_linear],
    "claim": [a_dynamic_joined],
    "leave": [a_static_1_percent],
    "pay": [a_increasing_linear],
    "vote": [a_dynamic_funded],
    "deploy": [a_normal],
    "cancel": [a_static_0_percent],
    
    # The hyperparameters of each action
    "join_stake": [h_join_stake],

    "pay_contribution": [h_pay_contribution],

    "vote_broker": [h_vote_broker],
    "vote_result": [h_vote_result],
    
    "deploy_initial_funds": [h_deploy_initial_funds],
}

## 3. Policy Functions

The policy functions carry out the actions of each agent and will return a dictionary that contains a summary or aggregate function of the actions that have taken place during the current epoch.

In [8]:
def p_free_memory(params, substep, state_history, previous_state):
    """Destroys the memory of some objects in the last timestep.
    
    https://community.cadcad.org/t/working-with-large-objects/215/2
    """
    if previous_state["timestep"] > 0:
        for variable in params["global"]:
            state = state_history[-1]
            
            for substate in state:
                substate[key] = None
                
    return dict()


def p_iter_epoch(params, substep, state_history, previous_state):
    for proposal in PROPOSALS.values():
        proposal.iter_epoch()
        
    print(f"Epoch: {previous_state['timestep']}")
        
    return dict()


def p_iter_features(params, substep, state_history, previous_state):
    for wallet in WALLETS.values():
        wallet.feature_vector += params["feature_noise"]()
        
    for proposal in PROPOSALS.values():
        proposal.feature_vector += params["feature_noise"]()
        
    return dict()


def p_join(params, substep, state_history, previous_state):
    n_joined = 0
    funds_staked = 0
    
    for wallet in WALLETS.values():
        for proposal in np.extract(params["join"](wallet, len(PROPOSALS), y_scale=0.05), list(PROPOSALS.values())):
            stake = params["join_stake"](wallet)
            
            wallet_funds_old = wallet.funds
            proposal_funds_old = proposal.funds + proposal.stake
            
            wallet = proposal.add_broker(wallet, stake)
            
            if wallet.funds != wallet_funds_old:
                transaction = {
                    "timestep": previous_state["timestep"],
                    "wallet": wallet.public,
                    "wallet_funds_old": wallet_funds_old,
                    "wallet_funds": wallet.funds,
                    "action": "join",
                    "proposal": proposal.public,
                    "proposal_funds_old": proposal_funds_old,
                    "proposal_funds": proposal.funds + proposal.stake,
                }

                TRANSACTIONS.append(transaction)

                n_joined += 1
                funds_staked += stake
            
    return {
        "n_joined": n_joined,
        "funds_staked": funds_staked,
    }
            
    
def p_claim(params, substep, state_history, previous_state):
    n_claimed = 0
    funds_claimed = 0
    
    for wallet in WALLETS.values():
        for proposal in np.extract(params["claim"](wallet, len(PROPOSALS), 1), list(PROPOSALS.values())):
            wallet_funds_old = wallet.funds
            proposal_funds_old = proposal.funds + proposal.stake
            
            wallet = proposal.claim_broker_funds(wallet)
            
            if wallet.funds != wallet_funds_old:
                transaction = {
                    "timestep": previous_state["timestep"],
                    "wallet": wallet.public,
                    "wallet_funds_old": wallet_funds_old,
                    "wallet_funds": wallet.funds,
                    "action": "claim",
                    "proposal": proposal.public,
                    "proposal_funds_old": proposal_funds_old,
                    "proposal_funds": proposal.funds + proposal.stake,
                }

                TRANSACTIONS.append(transaction)

                n_claimed += 1
                funds_claimed += wallet.funds - wallet_funds_old
            
    return {
        "n_claimed": n_claimed,
        "funds_claimed": funds_claimed,
    }


def p_leave(params, substep, state_history, previous_state):
    n_left = 0
    funds_claimed = 0
    
    for wallet in WALLETS.values():
        for proposal in np.extract(params["leave"](wallet, len(PROPOSALS)), list(PROPOSALS.values())):
            wallet_funds_old = wallet.funds
            proposal_funds_old = proposal.funds + proposal.stake
            
            wallet = proposal.remove_broker(wallet)
            
            if wallet.funds != wallet_funds_old:
                transaction = {
                    "timestep": previous_state["timestep"],
                    "wallet": wallet.public,
                    "wallet_funds_old": wallet_funds_old,
                    "wallet_funds": wallet.funds,
                    "action": "leave",
                    "proposal": proposal.public,
                    "proposal_funds_old": proposal_funds_old,
                    "proposal_funds": proposal.funds + proposal.stake,
                }

                TRANSACTIONS.append(transaction)

                n_left += 1
                funds_claimed += wallet.funds - wallet_funds_old
            
    return {
        "n_left": n_left,
        "funds_claimed": funds_claimed,
    }


def p_pay(params, substep, state_history, previous_state):
    n_paid = 0
    funds_contributed = 0

    for wallet in WALLETS.values():
        for proposal in np.extract(params["pay"](wallet, len(PROPOSALS), y_scale=0.01), list(PROPOSALS.values())):
            wallet_funds_old = wallet.funds
            proposal_funds_old = proposal.funds + proposal.stake
            
            contribution = params["pay_contribution"](wallet)
            
            wallet = proposal.pay(wallet, contribution)
            
            if wallet.funds != wallet_funds_old:
                transaction = {
                    "timestep": previous_state["timestep"],
                    "wallet": wallet.public,
                    "wallet_funds_old": wallet_funds_old,
                    "wallet_funds": wallet.funds,
                    "action": "pay",
                    "proposal": proposal.public,
                    "proposal_funds_old": proposal_funds_old,
                    "proposal_funds": proposal.funds + proposal.stake,
                }

                TRANSACTIONS.append(transaction)

                n_paid += 1
                funds_contributed += contribution
    
    return {
        "n_paid": n_paid,
        "funds_contributed": funds_contributed
    }


def p_vote(params, substep, state_history, previous_state):
    n_voted = 0
    voted_yes = 0
    
    for wallet in WALLETS.values():
        for proposal in np.extract(params["vote"](wallet, len(PROPOSALS), 1), list(PROPOSALS.values())):
            broker = WALLETS[params["vote_broker"](wallet)]
            result = params["vote_result"](wallet)
            
            proposal.vote_broker(wallet, broker, result)
            
            n_voted += 1
            voted_yes += result * 1
            
    return {
        "n_voted": n_voted,
        "voted_yes": voted_yes,
    }


def p_deploy(params, substep, state_history, previous_state):
    n_deployed = 0
    funds_contributed = 0
    
    for wallet in WALLETS.values():
        if params["deploy"](wallet, 1, y_scale=0.05)[0]:
            wallet_funds_old = wallet.funds
            initial_funds = params["deploy_initial_funds"](wallet)
            
            proposal = wallet.deploy(
                initial_funds=initial_funds,
                broker_whitelist=NoVote(),
                feature_vector=rng.random(size=wallet.number_of_features)
            )
            
            if proposal is not None:
                PROPOSALS[proposal.public] = proposal
                
                transaction = {
                    "timestep": previous_state["timestep"],
                    "wallet": wallet.public,
                    "wallet_funds_old": wallet_funds_old,
                    "wallet_funds": wallet.funds,
                    "action": "deploy",
                    "proposal": proposal.public,
                    "proposal_funds_old": 0,
                    "proposal_funds": proposal.funds,
                }
            
                TRANSACTIONS.append(transaction)
            
                n_deployed += 1
                funds_contributed += initial_funds
        
    return {
        "n_deployed": n_deployed,
        "funds_contributed": funds_contributed
    }


def p_cancel(params, substep, state_history, previous_state):
    n_cancelled = 0
    n_left = 0
    
    for wallet in WALLETS.values():
        for proposal in np.extract(params["cancel"](wallet, len(PROPOSALS)), list(PROPOSALS.values())):
            wallet_funds_old = wallet.funds
            proposal_funds_old = proposal.funds + proposal.stake
            
            proposal.cancel(wallet.public)
            
            for broker in list(proposal.broker_agreements.keys()):
                initial_funds = WALLETS[broker].funds
                WALLETS[broker] = proposal.remove_broker(WALLETS[broker])
                
                transaction = {
                    "timestep": previous_state["timestep"],
                    "wallet": broker,
                    "wallet_funds_old": wallet_funds_old,
                    "wallet_funds": WALLETS[broker].funds,
                    "action": "leave",
                    "proposal": proposal,
                    "proposal_funds_old": proposal_funds_old,
                    "proposal_funds": proposal.funds,
                }
            
                TRANSACTIONS.append(transaction)
                
                n_left += 1
                
            n_cancelled += 1
            
    return {
        "n_cancelled": n_cancelled,
        "n_left": n_left,
    }

## 4. State Update Functions

In [9]:
def s_total_funds(params, substep, state_history, previous_state, policy_input):
    wallet_funds = sum([wallet.funds for wallet in WALLETS.values()])
    proposal_funds = sum([proposal.funds + proposal.stake for proposal in PROPOSALS.values()])
    
    return "total_funds", wallet_funds + proposal_funds


def s_wallet_funds(params, substep, state_history, previous_state, policy_input):
    return "wallet_funds", sum([wallet.funds for wallet in WALLETS.values()])


def s_proposal_funds(params, substep, state_history, previous_state, policy_input):
    return "proposal_funds", sum([proposal.funds for proposal in PROPOSALS.values()])


def s_n_proposals(params, substep, state_history, previous_state, policy_input):
    return "n_proposals", len(PROPOSALS)


def s_owners(params, substep, state_history, previous_state, policy_input):
    n_owners = previous_state["owners"] + policy_input["n_deployed"] - policy_input["n_cancelled"]
    
    return "owners", n_owners


def s_payers(params, substep, state_history, previous_state, policy_input):
    n_payers = previous_state["payers"] + policy_input["n_paid"]
    
    return "payers", n_payers


def s_brokers(params, substep, state_history, previous_state, policy_input):
    n_brokers = previous_state["brokers"] + policy_input["n_joined"] - policy_input["n_left"]
    
    return "brokers", n_brokers


def s_funds_staked(params, substep, state_history, previous_state, policy_input):
    funds_staked = previous_state["funds_staked"] + policy_input["funds_staked"]
    
    return "funds_staked", funds_staked


def s_funds_claimed(params, substep, state_history, previous_state, policy_input):
    funds_claimed = previous_state["funds_claimed"] + policy_input["funds_claimed"]
    
    return "funds_claimed", funds_claimed


def s_funds_contributed(params, substep, state_history, previous_state, policy_input):
    funds_contributed = previous_state["funds_contributed"] + policy_input["funds_contributed"]
    
    return "funds_contributed", funds_contributed

## 5. Partial State Update Blocks

In [10]:
partial_state_update_blocks = [
    # Go to next epoch
    {
        "policies": {
            "iter_epoch": p_iter_epoch,
            "iter_features": p_iter_features,
        },
        "variables": {
        },
    },
    # Agents perform actions
    {   
        "policies": {
            "join": p_join,
            "claim": p_claim,
            "leave": p_leave,
            "pay": p_pay,
            "vote": p_vote,
            "deploy": p_deploy,
            "cancel": p_cancel,
        },
        'variables': {
            "total_funds": s_total_funds,
            "wallet_funds": s_wallet_funds,
            "proposal_funds": s_proposal_funds,
            "n_proposals": s_n_proposals,
            
            # Distribution of agent actions
            "owners": s_owners,
            "payers": s_payers,
            "brokers": s_brokers,
            
            # Proposal properties
            "funds_staked": s_funds_staked,
            "funds_claimed": s_funds_claimed,
            "funds_contributed": s_funds_contributed,
        }
    }
]

# Simulation

## 6. Configuration

In [11]:
sim_config = config_sim({
    "N": 1, # the number of times we'll run the simulation ("Monte Carlo runs")
    "T": range(100), # the number of timesteps the simulation will run for
    "M": system_params # the parameters of the system
})

In [12]:
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

## 7. Execution

In [13]:
exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (100, 13, 1, 11)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
Epoch: 0
Epoch: 1
Epoch: 2
Broker is not part of this proposal
Epoch: 3
Broker is not part of this proposal
Epoch: 4
Broker is not part of this proposal
Broker is not part of this proposal
Broker is not part of this proposal
Broker is not part of this proposal
Epoch: 5
Failed to add broker, broker already has a stake in this proposal
Failed to add broker, maximum number of brokers reached
Broker is not part of this proposal
Broker is not part of this proposal
Broker is not part of this proposal
Epoch: 6
Failed to add broker, p

Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, broker already has a stake in this proposal
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, broker already has a stake in this proposal
Failed to add broker, maximum number of brokers reached
Broker is not part of this proposal
Broker is not part of this proposal
Broker is no

Broker is not part of this proposal
Proposal has been cancelled, cannot add funds
Proposal has been cancelled, cannot add funds
Epoch: 22
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, broker already has a stake in this proposal
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, proposal has been cancelled
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, broker already has a stake in this proposal
Failed to add broker, broker already has a stake in this proposal
Failed to add broker, maximum number of brokers reache

Broker is not part of this proposal
Broker is not part of this proposal
Broker is not part of this proposal
Broker is not part of this proposal
Broker is not part of this proposal
Broker is not part of this proposal
Broker is not part of this proposal
Broker is not part of this proposal
Broker is not part of this proposal
Broker is not part of this proposal
Broker is not part of this proposal
Broker is not part of this proposal
Proposal has been cancelled, cannot add funds
Proposal has been cancelled, cannot add funds
Proposal has been cancelled, cannot add funds
Proposal has been cancelled, cannot add funds
Epoch: 28
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, proposal has been cancelled
Failed to add broker, broker already has a stake in this proposal
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum numb

Epoch: 32
Failed to add broker, proposal has been cancelled
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, broker already has a stake in this proposal
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, broker already has a stake in this proposal
Failed to add broker, maximum number of brokers reached
Failed to add broker, ma

...

Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, broker already has a stake in this proposal
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, proposal has been cancelled
Failed to add broker, maximum number of brokers reached
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, proposal has been cancelled
Failed to add broker, pr

Epoch: 97
Failed to add broker, proposal has been cancelled
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, proposal has been cancelled
Failed to add broker, maximum number of brokers reached
Failed to add broker, proposal has been cancelled
Failed to add broker, broker already has a stake in this proposal
Failed to add broker, proposal has been cancelled
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed t

Epoch: 98
Failed to add broker, proposal has been cancelled
Failed to add broker, maximum number of brokers reached
Failed to add broker, broker already has a stake in this proposal
Failed to add broker, maximum number of brokers reached
Failed to add broker, proposal has been cancelled
Failed to add broker, maximum number of brokers reached
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, maximum number of brokers reached
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, broker already has a stake in this proposal
Failed to add broker, proposal has been cancelled
Failed to add broker, maximum number of brokers reached
Failed to add broker, proposal has been cancelled
Failed to add broker, maximum number of brokers reached
Failed to add broker, 

Epoch: 99
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, proposal has been cancelled
Failed to add broker, maximum number of brokers reached
Failed to add broker, maximum number of brokers reached
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, broker already has a stake in this proposal
Failed to add broker, proposal has been cancelled
Failed to add broker, broker already has a stake in this proposal
Failed to add broker, maximum number of brokers reached
Failed to add broker, proposal has been cancelled
Failed to 

Total execution time: 12.21s


## 8. Output Preparation

What kind of metrics?

Can an agent of the simulation have multiple roles? For example, an owner who is also a broker, who is also a Payer?

Can track the wallets of all of the 'agents'. And the wallets ProposalInverter Contracts. 

Probably want some sort of proxy of the system like "Production". Can also have an idea of "Productivity". 

In [14]:
# Convert raw results to a Pandas DataFrame
df = pd.DataFrame(raw_result)

In [15]:
fdf = df[df["substep"] == 2]
fdf

,total_funds,wallet_funds,proposal_funds,n_wallets,n_proposals,owners,payers,brokers,funds_staked,funds_claimed,funds_contributed,simulation,subset,run,substep,timestep
2,500000.0,499900.0,100.0,50,1,1,0,0,0,0.0,100,0,0,1,2,1
4,500000.0,499650.0,350.0,50,3,3,1,0,0,0.0,350,0,0,1,2,2
6,500000.0,499540.0,450.0,50,4,4,1,1,10,0.0,450,0,0,1,2,3
8,500000.0,499430.0,540.0,50,4,4,3,3,30,10.0,550,0,0,1,2,4
10,500000.0,499360.0,570.0,50,4,4,4,7,70,30.0,600,0,0,1,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,500000.0,465050.0,28970.0,50,161,161,1133,723,8810,46610.0,72750,0,0,1,2,96
194,500000.0,464630.0,29320.0,50,164,164,1152,731,8900,47530.0,74000,0,0,1,2,97
196,500000.0,464200.0,29660.0,50,168,168,1173,742,9050,48700.0,75450,0,0,1,2,98
198,500000.0,463730.0,29960.0,50,170,170,1190,761,9300,49530.0,76500,0,0,1,2,99


In [16]:
# Insert cadCAD parameters for each configuration into DataFrame
for config in configs:
    # Get parameters from configuration
    parameters = config.sim_config['M']
    # Get subset index from configuration
    subset_index = config.subset_id
    
    # For each parameter key value pair
    for (key, value) in parameters.items():
        # Select all DataFrame indices where subset == subset_index
        dataframe_indices = df.eval(f'subset == {subset_index}')
        # Assign each parameter key value pair to the DataFrame for the corresponding subset
        df.loc[dataframe_indices, key] = value

df.head(10)

,total_funds,wallet_funds,proposal_funds,n_wallets,n_proposals,owners,payers,brokers,funds_staked,funds_claimed,...,leave,pay,vote,deploy,cancel,join_stake,pay_contribution,vote_broker,vote_result,deploy_initial_funds
0,500000.0,500000.0,0.0,50,0,0,0,0,0,0.0,...,<function a_static_1_percent at 0x000001E45189...,<function a_increasing_linear at 0x000001E4518...,<function a_dynamic_funded at 0x000001E4518E3040>,<function a_normal at 0x000001E4518E3280>,<function a_static_0_percent at 0x000001E45189...,<function h_join_stake at 0x000001E4518E38B0>,<function h_pay_contribution at 0x000001E4518E...,<function h_vote_broker at 0x000001E4518E39D0>,<function h_vote_result at 0x000001E4518E3A60>,<function h_deploy_initial_funds at 0x000001E4...
1,500000.0,500000.0,0.0,50,0,0,0,0,0,0.0,...,<function a_static_1_percent at 0x000001E45189...,<function a_increasing_linear at 0x000001E4518...,<function a_dynamic_funded at 0x000001E4518E3040>,<function a_normal at 0x000001E4518E3280>,<function a_static_0_percent at 0x000001E45189...,<function h_join_stake at 0x000001E4518E38B0>,<function h_pay_contribution at 0x000001E4518E...,<function h_vote_broker at 0x000001E4518E39D0>,<function h_vote_result at 0x000001E4518E3A60>,<function h_deploy_initial_funds at 0x000001E4...
2,500000.0,499900.0,100.0,50,1,1,0,0,0,0.0,...,<function a_static_1_percent at 0x000001E45189...,<function a_increasing_linear at 0x000001E4518...,<function a_dynamic_funded at 0x000001E4518E3040>,<function a_normal at 0x000001E4518E3280>,<function a_static_0_percent at 0x000001E45189...,<function h_join_stake at 0x000001E4518E38B0>,<function h_pay_contribution at 0x000001E4518E...,<function h_vote_broker at 0x000001E4518E39D0>,<function h_vote_result at 0x000001E4518E3A60>,<function h_deploy_initial_funds at 0x000001E4...
3,500000.0,499900.0,100.0,50,1,1,0,0,0,0.0,...,<function a_static_1_percent at 0x000001E45189...,<function a_increasing_linear at 0x000001E4518...,<function a_dynamic_funded at 0x000001E4518E3040>,<function a_normal at 0x000001E4518E3280>,<function a_static_0_percent at 0x000001E45189...,<function h_join_stake at 0x000001E4518E38B0>,<function h_pay_contribution at 0x000001E4518E...,<function h_vote_broker at 0x000001E4518E39D0>,<function h_vote_result at 0x000001E4518E3A60>,<function h_deploy_initial_funds at 0x000001E4...
4,500000.0,499650.0,350.0,50,3,3,1,0,0,0.0,...,<function a_static_1_percent at 0x000001E45189...,<function a_increasing_linear at 0x000001E4518...,<function a_dynamic_funded at 0x000001E4518E3040>,<function a_normal at 0x000001E4518E3280>,<function a_static_0_percent at 0x000001E45189...,<function h_join_stake at 0x000001E4518E38B0>,<function h_pay_contribution at 0x000001E4518E...,<function h_vote_broker at 0x000001E4518E39D0>,<function h_vote_result at 0x000001E4518E3A60>,<function h_deploy_initial_funds at 0x000001E4...
5,500000.0,499650.0,350.0,50,3,3,1,0,0,0.0,...,<function a_static_1_percent at 0x000001E45189...,<function a_increasing_linear at 0x000001E4518...,<function a_dynamic_funded at 0x000001E4518E3040>,<function a_normal at 0x000001E4518E3280>,<function a_static_0_percent at 0x000001E45189...,<function h_join_stake at 0x000001E4518E38B0>,<function h_pay_contribution at 0x000001E4518E...,<function h_vote_broker at 0x000001E4518E39D0>,<function h_vote_result at 0x000001E4518E3A60>,<function h_deploy_initial_funds at 0x000001E4...
6,500000.0,499540.0,450.0,50,4,4,1,1,10,0.0,...,<function a_static_1_percent at 0x000001E45189...,<function a_increasing_linear at 0x000001E4518...,<function a_dynamic_funded at 0x000001E4518E3040>,<function a_normal at 0x000001E4518E3280>,<function a_static_0_percent at 0x000001E45189...,<function h_join_stake at 0x000001E4518E38B0>,<function h_pay_contribution at 0x000001E4518E...,<function h_vote_broker at 0x000001E4518E39D0>,<function h_vote_result at 0x000001E4518E3A60>,<function h_deploy_initial_funds at 0x000001E4...
7,500000.0,499540.0,450.0,50,4,4,1,1,10,0.0,...,<function a_static_1_percent at 0x

In [17]:
transactions_df = pd.DataFrame(TRANSACTIONS)
transactions_df.to_csv("transactions.csv", header=True, index=False)
transactions_df

,timestep,wallet,wallet_funds,action,wallet_funds_old,proposal,proposal_funds_old,proposal_funds
0,0,0xD520BcB0B4030dEA628e362C9d7cb2684F39Cd0B,14460.177801,initialize,NaN,NaN,NaN,NaN
1,0,0xcE5C4C826C2E4099bE3333Bc8169dC52b0aa1418,8199.768299,initialize,NaN,NaN,NaN,NaN
2,0,0x1ce92474aF1519dE1d90D4f6760755aAEe034eed,16041.580920,initialize,NaN,NaN,NaN,NaN
3,0,0x130B967B2E07DFA842B3ACbD95795434Bd305E0f,13029.248510,initialize,NaN,NaN,NaN,NaN
4,0,0xf485D0ea1107D0b83683E32d5b7bDA4DDae26CDB,1759.558825,initialize,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
22283,100,0x9b3e6Df17CB67041041F6372449306bb255c066c,11562.209194,pay,11612.209194,0x8316Dc1Ae33D362Dc1065Fd169B503913f5a1b42,120.0,170.0
22284,100,0xC6FB0233fd090BE149Ac4ee69676ecFa94090060,13791.827038,pay,13841.827038,0x6EA24F43e7699591645C0121b07a80e3937D6A77,310.0,360.0
22285,100,0x4062EeF0B2f2802E1b5227B572F395F80559Cb69,6735.770592,pay,6785.770592,0x84Ca9f5FB90313c16C26fa2eA04b4DB3a6AF1014,300.0,350.0
22286,100,0x1ce92474aF1519dE1d90D4f6760755aAEe034eed,13819.914253,deploy,13919.914253,0x41Cba6855E4eE027a232bf8f3ce4Ef667294Bc16,0.0,100.0


## 9. Analysis

### 9.1 Total Funds

In [18]:
# timestep, wallet1, wallet2, ...
wfunds_df = pd.DataFrame({"timestep": sim_config[0]["T"]})

for public, wallet in WALLETS.items():
    wallet_df = transactions_df[transactions_df["wallet"] == public]
    wallet_df = wallet_df[["timestep", "wallet_funds"]]
    wallet_df = wallet_df.drop_duplicates(subset="timestep", keep="last")
    
    wallet_df.columns = ["timestep", wallet.public]
    wfunds_df = pd.merge(wfunds_df, wallet_df, how="outer", on="timestep")
    
wfunds_df = wfunds_df.fillna(method="ffill")
wfunds_df

,timestep,0xD520BcB0B4030dEA628e362C9d7cb2684F39Cd0B,0xcE5C4C826C2E4099bE3333Bc8169dC52b0aa1418,0x1ce92474aF1519dE1d90D4f6760755aAEe034eed,0x130B967B2E07DFA842B3ACbD95795434Bd305E0f,0xf485D0ea1107D0b83683E32d5b7bDA4DDae26CDB,0x45ce33CbfEf69a95DFe8A7BEdF5B9EFa1e2f76D4,0xd42612FFbF4B3B8ad3F58CB9CB9AA57BEffC0FE8,0xbA05D6DD5eA98680C248FfD9E0BE824cB48DB75c,0x5bf4907AD9a8fAe683CF656ffa7a7b4C12aDfb39,...,0x0Efee0cF19C82F2574b44F35403B8c02cA9f6B76,0x7b706b6d54212e8796E38758BFe1f4c6B26c7ab0,0x9b3e6Df17CB67041041F6372449306bb255c066c,0x6Fdf6629596Af98aF65eFB2D7BF8dd6537893Ed9,0xC6FB0233fd090BE149Ac4ee69676ecFa94090060,0xfe81827B822ABD89Ab69D461BDBcE2669497892a,0x4062EeF0B2f2802E1b5227B572F395F80559Cb69,0x5E91929ACDE1A54691940299F49f54CC599cea2C,0xfeA924f3B0bE1C4D78A58717A7501277FDC548c1,0xcC958B3172F9230e8D711630eeDfc4729843D583
0,0,14460.177801,8199.768299,16041.580920,13029.248510,1759.558825,18228.002349,14220.724087,14686.401945,2393.606085,...,8167.510913,15557.310765,13083.375860,5836.090023,15549.493705,15035.783645,7239.437259,5386.966942,12751.378265,2611.060105
1,1,14460.177801,8199.768299,16041.580920,13029.248510,1759.558825,18228.002349,14220.724087,14686.401945,2393.606085,...,8167.510913,15557.310765,13083.375860,5836.090023,15549.493705,15035.783645,7239.437259,5386.966942,12751.378265,2611.060105
2,2,14460.177801,8199.768299,16041.580920,13029.248510,1759.558825,18228.002349,14220.724087,14686.401945,2393.606085,...,8167.510913,15557.310765,13083.375860,5836.090023,15549.493705,15035.783645,7239.437259,5386.966942,12751.378265,2611.060105
3,3,14460.177801,8199.768299,16041.580920,13029.248510,1759.558825,18228.002349,14220.724087,14686.401945,2383.606085,...,8167.510913,15557.310765,13083.375860,5836.090023,15549.493705,15035.783645,7239.437259,5386.966942,12751.378265,2611.060105
4,4,14460.177801,8199.768299,16041.580920,13029.248510,1759.558825,18228.002349,14220.724087,14686.401945,2393.606085,...,8167.510913,15557.310765,13083.375860,5826.090023,15549.493705,14985.783645,7239.437259,5386.966942,12751.378265,2611.060105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,96,13514.011134,8114.101632,14053.414253,11495.081844,3457.058825,15389.002349,12720.724087,13438.401945,3018.439418,...,7638.010913,13674.144098,11628.209194,5927.090023,13875.827038,13522.616979,6883.770592,5553.133608,11879.711598,3446.893438
97,97,13474.011134,8124.601632,14007.414253,11501.581844,3443.058825,15339.002349,12731.224087,13442.401945,3023.439418,...,7650.510913,13682.644098,11634.209194,5895.590023,13829.827038,13526.616979,6896.270592,5569.133608,11889.711598,3490.893438
98,98,13384.011134,8151.101632,13911.414253,11508.081844,3489.558825,15339.002349,12691.224087,13460.401945,3069.272751,...,7663.010913,13708.644098,11644.209194,5854.090023,13833.827038,13430.616979,6798.770592,5595.133608,11900.211598,3509.393438
99,99,13394.011134,8114.934966,13915.914253,11504.081844,3536.058825,15339.002349,12651.224087,13462.401945,2993.272751,...,7685.510913,13664.644098,11653.209194,5852.590023,13837.827038,13435.116979,6770.770592,5572.133608,11910.211598,3530.393438


In [36]:
wfunds_df.plot(x="timestep", y=list(WALLETS.keys()), title="Wallet Funds")

In [20]:
# Positions show how much each wallet made relative to their initial value
wpositions_df = wfunds_df.copy()

initial_funds = np.array(wpositions_df[wfunds_df["timestep"] == 0][list(WALLETS.keys())])
wpositions_df[list(WALLETS.keys())] = wpositions_df[list(WALLETS.keys())].sub(initial_funds)

wpositions_df

,timestep,0xD520BcB0B4030dEA628e362C9d7cb2684F39Cd0B,0xcE5C4C826C2E4099bE3333Bc8169dC52b0aa1418,0x1ce92474aF1519dE1d90D4f6760755aAEe034eed,0x130B967B2E07DFA842B3ACbD95795434Bd305E0f,0xf485D0ea1107D0b83683E32d5b7bDA4DDae26CDB,0x45ce33CbfEf69a95DFe8A7BEdF5B9EFa1e2f76D4,0xd42612FFbF4B3B8ad3F58CB9CB9AA57BEffC0FE8,0xbA05D6DD5eA98680C248FfD9E0BE824cB48DB75c,0x5bf4907AD9a8fAe683CF656ffa7a7b4C12aDfb39,...,0x0Efee0cF19C82F2574b44F35403B8c02cA9f6B76,0x7b706b6d54212e8796E38758BFe1f4c6B26c7ab0,0x9b3e6Df17CB67041041F6372449306bb255c066c,0x6Fdf6629596Af98aF65eFB2D7BF8dd6537893Ed9,0xC6FB0233fd090BE149Ac4ee69676ecFa94090060,0xfe81827B822ABD89Ab69D461BDBcE2669497892a,0x4062EeF0B2f2802E1b5227B572F395F80559Cb69,0x5E91929ACDE1A54691940299F49f54CC599cea2C,0xfeA924f3B0bE1C4D78A58717A7501277FDC548c1,0xcC958B3172F9230e8D711630eeDfc4729843D583
0,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,-10.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,-10.0,0.000000,-50.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,96,-946.166667,-85.666667,-1988.166667,-1534.166667,1697.5,-2839.0,-1500.0,-1248.0,624.833333,...,-529.5,-1883.166667,-1455.166667,91.0,-1673.666667,-1513.166667,-355.666667,166.166667,-871.666667,835.833333
97,97,-986.166667,-75.166667,-2034.166667,-1527.666667,1683.5,-2889.0,-1489.5,-1244.0,629.833333,...,-517.0,-1874.666667,-1449.166667,59.5,-1719.666667,-1509.166667,-343.166667,182.166667,-861.666667,879.833333
98,98,-1076.166667,-48.666667,-2130.166667,-1521.166667,1730.0,-2889.0,-1529.5,-1226.0,675.666667,...,-504.5,-1848.666667,-1439.166667,18.0,-1715.666667,-1605.166667,-440.666667,208.166667,-851.166667,898.333333
99,99,-1066.166667,-84.833333,-2125.666667,-1525.166667,1776.5,-2889.0,-1569.5,-1224.0,599.666667,...,-482.0,-1892.666667,-1430.166667,16.5,-1711.666667,-1600.666667,-468.666667,185.166667,-841.166667,919.333333


In [21]:
wpositions_df.plot(x="timestep", y=list(WALLETS.keys()))

In [22]:
# timestep, proposal1, proposal2, ...
pfunds_df = pd.DataFrame({"timestep": sim_config[0]["T"]})

for public, proposal in PROPOSALS.items():
    proposal_df = transactions_df[transactions_df["proposal"] == public]
    proposal_df = proposal_df[["timestep", "proposal_funds"]]
    proposal_df = proposal_df.drop_duplicates(subset="timestep", keep="last")
    
    proposal_df.columns = ["timestep", proposal.public]
    pfunds_df = pd.merge(pfunds_df, proposal_df, how="outer", on="timestep")
    
pfunds_df = pfunds_df.fillna(method="ffill")
pfunds_df

,timestep,0x37B5ed5C8bE3138d0Cf70A9D5e4922f19352AcCE,0xC32a6Af474b7132Ae2E5E753e203123F99eF7da2,0xBC6BbaA8ff7FA50b17e16D9164a15742eC2450Da,0xdAc7A4f91c1b93b69Adb9143489C7562BB4E520F,0x78ab0e6e9c86B5232780fab8ED1a3552728f1D39,0xa15c170A5fdF7D4f63325c6209ca3c0cfE674CE5,0xD0e5A366842fC1d9DaE2ef18ED8756D01b125067,0x5d5e842c30590B993aC9AF7eE299D6d5fd9A6724,0xEcbC7a5fEf87599D6071f4C7ed84E6A5777B347e,...,0x6D0BE73829eDF9191b4D0eb0BF4572AB3958bdC5,0x03898e11F7C0e3079F02E9cE94326bD0Ee37E53b,0xFA8d514d36Eb162c288Ffe19D4295676671Cc3B1,0x99065bd0a9D9B5fe61b1f4DC8178d6D378D871C2,0x584Da5738ac430bd35a89484E4a25f0d67D80a6b,0x82307789B08d8061e79cB7f3275F24d704F252b5,0xFC5805d36402B6374236823b9D40F3C986cF5CC5,0x8316Dc1Ae33D362Dc1065Fd169B503913f5a1b42,0x41Cba6855E4eE027a232bf8f3ce4Ef667294Bc16,0xc63E9B13f3b6E476EC1042A25d5E2Dcb1553Fcd0
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,150.0,100.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,150.0,110.0,100.0,100.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,200.0,110.0,150.0,110.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,96,890.0,10.0,420.0,30.0,320.0,680.0,640.0,20.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,97,880.0,10.0,410.0,30.0,360.0,670.0,630.0,20.0,10.0,...,100.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,98,920.0,10.0,400.0,30.0,360.0,660.0,620.0,20.0,10.0,...,120.0,100.0,100.0,100.0,100.0,100.0,NaN,NaN,NaN,NaN
99,99,900.0,10.0,440.0,30.0,350.0,650.0,660.0,20.0,10.0,...,110.0,110.0,110.0,130.0,160.0,140.0,100.0,100.0,NaN,NaN


In [23]:
pfunds_df.plot(x="timestep", y=[key for key, value in PROPOSALS.items() if value.funds > 0])

In [24]:
pfunds_df.plot(x="timestep", y=[key for key, value in PROPOSALS.items() if value.funds <= 0])

In [25]:
# Positions show how many funds each proposal has relative to its minimum horizon
ppositions_df = pfunds_df.copy()

min_horizons = pd.DataFrame(
    data=[[proposal.min_horizon * proposal.allocation_per_epoch for proposal in PROPOSALS.values()]],
    columns=list(PROPOSALS.keys())
)
# Why does this give nans?
#ppositions_df[list(PROPOSALS.keys())] = ppositions_df[list(PROPOSALS.keys())].sub(min_horizons)
ppositions_df[list(PROPOSALS.keys())] = ppositions_df[list(PROPOSALS.keys())].sub(70)

ppositions_df

,timestep,0x37B5ed5C8bE3138d0Cf70A9D5e4922f19352AcCE,0xC32a6Af474b7132Ae2E5E753e203123F99eF7da2,0xBC6BbaA8ff7FA50b17e16D9164a15742eC2450Da,0xdAc7A4f91c1b93b69Adb9143489C7562BB4E520F,0x78ab0e6e9c86B5232780fab8ED1a3552728f1D39,0xa15c170A5fdF7D4f63325c6209ca3c0cfE674CE5,0xD0e5A366842fC1d9DaE2ef18ED8756D01b125067,0x5d5e842c30590B993aC9AF7eE299D6d5fd9A6724,0xEcbC7a5fEf87599D6071f4C7ed84E6A5777B347e,...,0x6D0BE73829eDF9191b4D0eb0BF4572AB3958bdC5,0x03898e11F7C0e3079F02E9cE94326bD0Ee37E53b,0xFA8d514d36Eb162c288Ffe19D4295676671Cc3B1,0x99065bd0a9D9B5fe61b1f4DC8178d6D378D871C2,0x584Da5738ac430bd35a89484E4a25f0d67D80a6b,0x82307789B08d8061e79cB7f3275F24d704F252b5,0xFC5805d36402B6374236823b9D40F3C986cF5CC5,0x8316Dc1Ae33D362Dc1065Fd169B503913f5a1b42,0x41Cba6855E4eE027a232bf8f3ce4Ef667294Bc16,0xc63E9B13f3b6E476EC1042A25d5E2Dcb1553Fcd0
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,80.0,30.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,80.0,40.0,30.0,30.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,130.0,40.0,80.0,40.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,96,820.0,-60.0,350.0,-40.0,250.0,610.0,570.0,-50.0,-60.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,97,810.0,-60.0,340.0,-40.0,290.0,600.0,560.0,-50.0,-60.0,...,30.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,98,850.0,-60.0,330.0,-40.0,290.0,590.0,550.0,-50.0,-60.0,...,50.0,30.0,30.0,30.0,30.0,30.0,NaN,NaN,NaN,NaN
99,99,830.0,-60.0,370.0,-40.0,280.0,580.0,590.0,-50.0,-60.0,...,40.0,40.0,40.0,60.0,90.0,70.0,30.0,30.0,NaN,NaN


In [26]:
ppositions_df.plot(x="timestep", y=[key for key, value in PROPOSALS.items() if value.funds > 0])

In [27]:
ppositions_df.plot(x="timestep", y=[key for key, value in PROPOSALS.items() if value.funds <= 0])

In [28]:
broker_holdings = dict()

for index, transaction in transactions_df.iterrows():
    if transaction["action"] == "claim":
        wallet = transaction["wallet"]
        
        if wallet not in broker_holdings.keys():
            broker_holdings[wallet] = {0: 0}
            
        broker_holdings[wallet][transaction["timestep"]] = max(broker_holdings[wallet].values()) + (transaction["wallet_funds"] - transaction["wallet_funds_old"])

broker_holdings_df = pd.DataFrame(broker_holdings)
broker_holdings_df = broker_holdings_df.sort_index()
broker_holdings_df = broker_holdings_df.fillna(method="ffill")
broker_holdings_df

,0x5bf4907AD9a8fAe683CF656ffa7a7b4C12aDfb39,0x137B3a2122f3652d1DAd6028FFF313049B93ac9D,0x6Fdf6629596Af98aF65eFB2D7BF8dd6537893Ed9,0x265a7ba872D71361D58de0d54B4DD7A0EfF685ae,0x04ABA123439CdAa0f511f3d77db47D796c1e0429,0xf4A6a59954b1A40c68A749B359f46d565D5807A8,0x05c49AB68a390BDc90Fd67Cece86Bf9Ea99aCBf4,0x366bf01F9AA5068EBB8d4834a38637E10F97c7Ee,0x2d4E7d8cFeaae909B4c9b8e4a9B507B751C1437b,0x34f3D6CdCA5b7293db2F669750048D5159D9ba44,...,0x93220eB8487a786CbB2d9a4fff9223DF952A3F96,0xD520BcB0B4030dEA628e362C9d7cb2684F39Cd0B,0x4049Ed8d97042F7e83d1FFbfe287d90e5a5472f9,0x9b3e6Df17CB67041041F6372449306bb255c066c,0x7b706b6d54212e8796E38758BFe1f4c6B26c7ab0,0x76Ad1506039FAAFD15Cd1232eF94C70b71ec8D28,0xC6FB0233fd090BE149Ac4ee69676ecFa94090060,0xfe81827B822ABD89Ab69D461BDBcE2669497892a,0x9811C6d479e10595Ca9c7067ebD8281F781E7bD2,0x45ce33CbfEf69a95DFe8A7BEdF5B9EFa1e2f76D4
0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,10.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
5,15.000000,5.000000,10.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
6,18.333333,8.333333,15.0,5.000000,5.000000,5.000000,3.333333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
7,38.333333,28.333333,17.5,7.000000,7.500000,7.000000,23.333333,2.000000,2.000000,2.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1574.833333,1816.666667,1181.0,2009.500000,1480.666667,1068.333333,1271.166667,1400.333333,1650.166667,1401.666667,...,1191.166667,473.833333,625.0,294.833333,426.833333,474.166667,346.333333,216.833333,174.0,81.0
97,1599.833333,1884.833333,1199.5,2055.000000,1507.166667,1087.333333,1336.833333,1424.333333,1694.166667,1427.666667,...,1209.666667,483.833333,651.0,300.833333,435.333333,482.666667,350.333333,220.833333,178.0,81.0
98,1665.666667,1939.333333,1218.0,2084.000000,1566.166667,1119.833333,1364.833333,1448.333333,1736.166667,1465.666667,...,1228.166667,493.833333,675.0,320.833333,461.333333,490.666667,354.333333,224.833333,182.0,81.0
99,1699.666667,1975.833333,1246.5,2114.000000,1591.500000,1135.833333,1392.666667,1477.833333,1762.166667,1488.166667,...,1246.666667,503.833333,685.5,329.833333,467.333333,498.666667,358.333333,229.333333,186.0,81.0


In [29]:
broker_holdings_df.plot(kind='area')

In [34]:
for proposal in PROPOSALS.values():
    if len(proposal.broker_agreements) >= 5 and not proposal.cancelled:
        proposal_id = proposal.public
        break

broker_holdings = dict()

for index, transaction in transactions_df.iterrows():
    if transaction["proposal"] == proposal_id and transaction["action"] == "claim":
        wallet = transaction["wallet"]
        
        if wallet not in broker_holdings.keys():
            broker_holdings[wallet] = {0: 0}
            
        broker_holdings[wallet][transaction["timestep"]] = max(broker_holdings[wallet].values()) + (transaction["wallet_funds"] - transaction["wallet_funds_old"])

proposal_holdings_df = pd.DataFrame(broker_holdings)
proposal_holdings_df = proposal_holdings_df.sort_index()
proposal_holdings_df = proposal_holdings_df.fillna(method="ffill")
proposal_holdings_df

,0x265a7ba872D71361D58de0d54B4DD7A0EfF685ae,0xf4A6a59954b1A40c68A749B359f46d565D5807A8,0x366bf01F9AA5068EBB8d4834a38637E10F97c7Ee,0x2d4E7d8cFeaae909B4c9b8e4a9B507B751C1437b,0x34f3D6CdCA5b7293db2F669750048D5159D9ba44,0xcE5C4C826C2E4099bE3333Bc8169dC52b0aa1418,0x7b706b6d54212e8796E38758BFe1f4c6B26c7ab0,0x0Efee0cF19C82F2574b44F35403B8c02cA9f6B76,0x9811C6d479e10595Ca9c7067ebD8281F781E7bD2
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7.0,7.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
8,9.0,9.0,4.0,4.0,4.0,0.0,0.0,0.0,0.0
9,11.0,11.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
96,74.0,170.5,111.0,93.0,77.0,149.5,137.5,18.0,79.5
97,76.0,172.5,111.0,95.0,77.0,149.5,139.5,18.0,81.5
98,78.0,174.5,111.0,97.0,77.0,149.5,141.5,18.0,83.5
99,80.0,176.5,111.0,99.0,77.0,149.5,143.5,18.0,85.5


In [37]:
proposal_holdings_df.plot(kind='area', title="Claims In A Single Proposal")